# 1. Importing data

In [1]:
import pandas as pd
import os
os.chdir(r'\Users\Sacha.dolle\Desktop\Cours IH\Week 2\project')
attacks=pd.read_csv('attacks.csv', sep=',', encoding ='latin1')

# 2. Cleaning data

In [2]:
def clean_col(attacks):

    attacks=attacks[['Date', 'Year', 'Type', 'Country', 'Area', 'Location','Activity', 'Name', 'Sex ', 'Age', 'Fatal (Y/N)', 'Time', 'Species ']]

    attacks=attacks.rename(columns={'Fatal (Y/N)' : 'fatal'})
    fatal_error=attacks.loc[~(attacks.fatal=='Y')&~(attacks.fatal=='y')]
    attacks=attacks.drop(fatal_error.index, axis=0)

    not_last_years=attacks.loc[~(attacks.Year>=1980)]
    attacks=attacks.drop(not_last_years.index, axis=0)

    attacks=attacks.drop('fatal', axis=1)
    
    return attacks


#Drops columns with more than 25% of missing values
def missing_value(attacks):
    null_cols=attacks.isnull().sum()
    per_null_cols=null_cols[null_cols>0]/attacks.shape[0]*100 
    ind_null_cols=per_null_cols[per_null_cols>25].index
    attacks = attacks.drop(ind_null_cols, axis=1)
    attacks=attacks.fillna('0')
    return attacks

def type_col(attacks):
# There is too many objetcs
    attacks.Year=attacks.Year.astype('int')
    attacks.Activity=attacks.Activity.astype('str')
    attacks.Activity.str.lower()
    attacks.Country=attacks.Country.str.title()
    return attacks

def text(attacks):
    attacks.Area=attacks.Area.str.replace('-', ' ')
    attacks.Area=attacks.Area.str.replace('()', '')
    attacks.Area=attacks.Area.str.replace('\x92', '\'')
    attacks.Area=attacks.Area.str.replace('0', 'Unknown data')
    attacks.Area=attacks.Area.str.replace('St.', 'Saint')

    attacks.Location=attacks.Location.str.replace('-', ' ')
    attacks.Location=attacks.Location.str.replace('()', '')
    attacks.Location=attacks.Location.str.replace('\x92', '\'')
    attacks.Location=attacks.Location.str.replace('0', 'Unknown data')
    attacks.Location=attacks.Location.str.replace('St.', 'Saint')

    attacks.Name=attacks.Name.str.replace('\(female\)', '')
    attacks.Name=attacks.Name.str.replace('  ', '')
    attacks.Name=attacks.Name.str.replace('____', '')

    attacks.iloc[ : , 8]=attacks.iloc[ : , 8].str.replace('0', 'NA')


    attacks.loc[attacks.Activity.str.contains('capsized'), 'Activity']='Sinking'
    attacks.loc[attacks.Activity.str.contains('sinking'), 'Activity']='Sinking'
    attacks.loc[attacks.Activity.str.contains('sank'), 'Activity']='Sinking'
    attacks.loc[attacks.Activity.str.contains('diving'), 'Activity']='Diving'
    attacks.loc[attacks.Activity.str.contains('fishing'), 'Activity']='Fishing'
    attacks.loc[attacks.Activity.str.contains('surf'), 'Activity']='Surfing'
    attacks.loc[attacks.Activity.str.contains('snorkeling'), 'Activity']='Snorkeling'
    attacks.loc[attacks.Activity.str.contains('swimming'), 'Activity']='Swimming'
    attacks.loc[attacks.Activity.str.contains('bath'), 'Activity']='Bathing'
    attacks.loc[attacks.Activity.str.contains('disaster'), 'Activity']='Disaster'
    attacks.loc[attacks.Activity.str.contains('paddl'), 'Activity']='Paddling'
    attacks.loc[attacks.Activity.str.contains('board'), 'Activity']='Boarding'
    attacks.loc[attacks.Activity.str.contains('0'), 'Activity']='Not specified'


    clusters=['Swimming', 'Diving', 'Sinking', 'Bathing', 'Snorkeling', 'Boarding', 'Surfing', 'Walking', 'Disaster', 'Fishing', 'Not specified', 'Standing', 'Paddling']

    attacks.loc[~attacks.Activity.str.contains('|'.join(clusters)), 'Activity']='Other'
    
    return attacks

In [17]:
def cleaning(attacks):
    clean_col(attacks)
    missing_value(attacks)
    type_col(attacks)
    text(attacks)
    return attacks

In [3]:
clean_col(attacks)

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Time,Species
6,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,Late afternoon,Tiger shark
58,30-Nov-2017,2017.0,Unprovoked,COSTA RICA,Cocos Island,Manuelita,Scuba diving,Rohina Bhandari,F,49,NaN,"Tiger shark, female"
59,Reported 25-Nov-2017,2017.0,Sea Disaster,LIBYA,NaN,Gars Garabulli,2 boats capsized,31 migrants,NaN,NaN,NaN,Some drowned but other may have been killed by...
64,04-Nov-2017,2017.0,Unprovoked,CUBA,Holquin Province,Guardalavaca Beach,Night bathing,Jesús Cabrera González,M,22,23h30,Tiger shark
91,03-Sep-2017,2017.0,Unprovoked,SOUTH AFRICA,Western Cape Province,Dyer Island,"Swimming, poaching abalone",Sivuyile Xelela,M,NaN,Morning,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3056,Reported 22-Aug-1980,1980.0,Unprovoked,MEXICO,NaN,NaN,Diving in tuna net,Gerald Correria,M,22,NaN,NaN
3067,22-Apr-1980,1980.0,Sea Disaster,PHILIPPINES,Romblon Province,"Maestre de Campo Island, 120 miles south of Ma...",Sinking of the ferryboat Don Juan,NaN,NaN,NaN,Night,NaN
3069,29-Mar-1980,1980.0,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Umdhloti,Free diving,Ray Booth,M,52,NaN,Tiger shark
3071,15-Mar-1980,1980.0,Sea Disaster,PHILIPPINES,NaN,250 miles southwest of Manila,Sinking of the ferryboat Bongbong 1,2 women,F,NaN,NaN,NaN


In [4]:
missing_value(attacks)

""
0
1
2
3
4
...
25718
25719
25720
25721


In [5]:
type_col(attacks)

ValueError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
text(attacks)